# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

https://github.com/datajoint/workflow-calcium-imaging/blob/main/notebooks/03-process.ipynb
https://github.com/datajoint/workflow-calcium-imaging/blob/main/notebooks/04-automate-optional.ipynb

Check the Moser lab descriptions of useage as well.
https://github.com/kavli-ntnu/dj-docs/blob/master/notebooks/2022-11%20Imaging%20workshop/Working_with_Imaging_pipeline.ipynb

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, event, trial, imaging
from adamacs import utility
from adamacs.ingest import behavior as ibe
import numpy as np
import scanreader

[2023-02-25 21:24:26,026][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-02-25 21:24:26,087][INFO]: Connected tobiasr@172.26.128.53:3306


Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'root'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, event, trial, imaging
from adamacs import utility
from adamacs.ingest import behavior as ibe
import numpy as np

## populate scaninfo

In [ ]:
scan.ScanInfo.describe()

In [ ]:
scan.ScanInfo.heading

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(scan) + dj.Diagram(imaging.Processing)

In [ ]:
scan.ScanInfo.delete()
scan.Scan.delete()
session.Session.delete()

In [ ]:
scan.Scan() * scan.ScanPath() * session.Session() * scan.ScanLocation()

## populate scan!

In [35]:
populate_settings = {'display_progress': True}

In [ ]:
scan.ScanInfo.populate(**populate_settings)
scan.ScanInfo()

In [ ]:
scan.Scan() * scan.ScanLocation()

In [ ]:
scan.ScanInfo()

Example to update an entry post-hoc. TODO: imnplement in elements "Scan" function based on userfunction stringa

In [ ]:
scan.Scan.update1({'session_id': 'sess9FF6TL96', 'scan_id': 'scan9FF6TL96', 'scan_notes': "test2"})

In [ ]:
scan.ScanLocation.update1({'session_id': 'sess9FF6TL96', 'scan_id': 'scan9FF6TL96', 'anatomical_location': "V1"})

In [ ]:
session.ProjectSession()

In [ ]:
scan.Scan()

In [ ]:
scan.ScanLocation()

### Creating a Parameter Set

What exactly happens during processing dependso on the parameter set. This is an example of a parameter set and its insert:

In [3]:
dj.config['custom'].get('suite2p_fast_tmp')[0]

'/home/tobiasr/tmp/'

In [2]:
# Insert the param_set

# TODO: Parameter set needs to be updated with ScanInfo settings

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [3]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=params_suite2p,
    paramset_desc="Mini2p (single channel, single plane, 15Hz)",
)


In [4]:
imaging.ProcessingParamSet()

paramset_idx,processing_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,suite2p,"Mini2p (single channel, single plane, 15Hz)",04677ecd-83f1-6298-acd9-ce8fa775e7a7,=BLOB=


In [2]:
imaging.ProcessingTask().delete()
imaging.ProcessingTask().drop()
imaging.ProcessingParamSet().drop()

[2023-02-25 21:23:38,300][INFO]: Deleting 2 rows from `tobiasr_imaging`.`processing_task`
[2023-02-25 21:23:40,862][INFO]: Deletes committed.
[2023-02-25 21:23:40,902][INFO]: `tobiasr_imaging`.`processing_task` (0 tuples)
[2023-02-25 21:23:40,903][INFO]: `tobiasr_imaging`.`__processing` (0 tuples)
[2023-02-25 21:23:40,906][INFO]: `tobiasr_imaging`.`curation` (0 tuples)
[2023-02-25 21:23:40,907][INFO]: `tobiasr_imaging`.`__segmentation` (0 tuples)
[2023-02-25 21:23:40,910][INFO]: `tobiasr_imaging`.`__segmentation__mask` (0 tuples)
[2023-02-25 21:23:40,912][INFO]: `tobiasr_imaging`.`_motion_correction` (0 tuples)
[2023-02-25 21:23:40,914][INFO]: `tobiasr_imaging`.`_motion_correction__non_rigid_motion_correction` (0 tuples)
[2023-02-25 21:23:40,916][INFO]: `tobiasr_imaging`.`_motion_correction__rigid_motion_correction` (0 tuples)
[2023-02-25 21:23:40,918][INFO]: `tobiasr_imaging`.`_motion_correction__summary` (0 tuples)
[2023-02-25 21:23:40,920][INFO]: `tobiasr_imaging`.`_motion_correctio

### Create and Run a Processing Task

In [5]:
scanquey = 'scan9FGLZLRI'

query =  scan.ScanInfo() & 'scan_id = "' + scanquey + '"'
sess_proc = query.fetch('session_id')[0]
scan_proc = query.fetch('scan_id')[0]

query2 = session.SessionDirectory() & 'session_id = "' + query.fetch('session_id')[0] + '"'
dir_proc = query2.fetch('session_dir')[0]

In [6]:
imaging.ProcessingTask.insert1((sess_proc,
                                scan_proc,
                                0,
                                dir_proc,
                                'trigger'))

In [33]:
imaging.ProcessingTask()

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
sess9FGLZLRI,scan9FGLZLRI,0,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,trigger


To run all unprocessed processing task we call populate on processing:

In [8]:
imaging.Processing.populate(display_progress=True)

Processing:   0%|          | 0/1 [00:00<?, ?it/s]

using TorchFFT
{'data_path': ['/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI'], 'tiff_list': ['/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI/scan9FGLZLRI_RN_OPI-1681_00001.tif']}
tif
** Found 1 tifs - converting to binary **
time 1.58 sec. Wrote 5000 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: Applying builtin classifier at /home/tobiasr/suite2p/suite2p/classifiers/classifier.npy
----------- REGISTRATION
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 5.68 sec.
Registered 5000/5000 in 20.57s
----------- Total 31.42 sec
Registration metrics, 10.60 sec.
----------- ROI DETECTION
Binning movie in chunks of length 15
Binned movie of size [332,216,226] created in 0.71 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.00, threshold 5.00 
0 ROI

/home/tobiasr/miniconda3/envs/datajoint/lib/python3.8/site-packages/scipy/io/matlab/_mio5.py:493: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  narr = np.asanyarray(source)
Processing: 100%|██████████| 1/1 [00:57<00:00, 57.53s/it]


In [47]:
for key in (imaging.ProcessingTask - imaging.Curation).fetch('KEY'):
    imaging.Curation().create1_from_processing_task(key)

In [53]:
imaging.Curation()

session_id,scan_id,paramset_idx,curation_id,curation_time time of generation of this set of curated results,"curation_output_dir output directory of the curated results, relative to root data directory",manual_curation has manual curation been performed on this result?,curation_note
sess9FGLZLRI,scan9FGLZLRI,0,1,2023-02-25 17:22:01,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,0,


In [54]:
imaging.MotionCorrection.populate(**populate_settings)

MotionCorrection:   0%|          | 0/1 [00:00<?, ?it/s]


KeyError: 'nblocks'

In [55]:
imaging.Segmentation.populate(**populate_settings)

Segmentation: 100%|██████████| 1/1 [00:00<00:00, 12.24it/s]


In [56]:
imaging.MaskClassification.populate(**populate_settings)

MaskClassification: 100%|██████████| 1/1 [00:00<00:00, 828.42it/s]


In [57]:
imaging.Fluorescence.populate(**populate_settings)

Fluorescence: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]


In [58]:
imaging.Activity.populate(**populate_settings)

Activity: 100%|██████████| 1/1 [00:00<00:00, 12.59it/s]


AttributeError: 'Activity' object has no attribute 'count'

In [62]:
imaging.ProcessingTask * imaging.Processing & session_key

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation","processing_time time of generation of this set of processed, segmented results",package_version
sess9FGLZLRI,scan9FGLZLRI,0,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,trigger,2023-02-25 17:22:01,


In [10]:
imaging.ProcessingTask()

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
sess9FGLZLRI,scan9FGLZLRI,0,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,trigger


In [ ]:
imaging.Processing()

In [13]:
session_key = (session.Session & 'subject = "OPI-1681"').fetch('KEY')[0]

In [14]:
scan.ScanInfo.ScanFile()


session_id,scan_id,file_path filepath relative to root data directory
sess9FGLZLRI,scan9FGLZLRI,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI/scan9FGLZLRI_RN_OPI-1681_00001.tif


In [15]:
scan.Scan & session_key

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FGLZLRI,scan9FGLZLRI,mini2p1,ScanImage,


In [16]:
scan.ScanInfo & session_key

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FGLZLRI,scan9FGLZLRI,1,1,1,5000,0,nan,nan,nan,15.3845,1,250.001,0.95,None,325.002


In [17]:
scan.ScanInfo.Field & session_key

session_id,scan_id,field_idx,px_height height in pixels,px_width width in pixels,um_height height in microns,um_width width in microns,field_x (um) center of field in the motor coordinate system,field_y (um) center of field in the motor coordinate system,field_z (um) relative depth of field,delay_image (ms) delay between the start of the scan and pixels in this field,roi the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
sess9FGLZLRI,scan9FGLZLRI,0,256,256,164.491,164.491,nan,nan,nan,=BLOB=,None


In [18]:
imaging.ProcessingParamSet()

paramset_idx,processing_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,suite2p,"Mini2p (single channel, single plane, 15Hz)",04677ecd-83f1-6298-acd9-ce8fa775e7a7,=BLOB=


In [19]:
imaging.ProcessingTask * imaging.Processing & session_key

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation","processing_time time of generation of this set of processed, segmented results",package_version
sess9FGLZLRI,scan9FGLZLRI,0,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,trigger,2023-02-25 17:22:01,


In [20]:
imaging.Curation & session_key

session_id,scan_id,paramset_idx,curation_id,curation_time time of generation of this set of curated results,"curation_output_dir output directory of the curated results, relative to root data directory",manual_curation has manual curation been performed on this result?,curation_note


Scanreader payload testing

In [21]:
path = scan.ScanInfo.ScanFile().fetch('file_path')[0]
infoscan = scanreader.read_scan(path)

In [25]:
infoscan.fpd

AttributeError: 'Scan2022' object has no attribute 'fpd'

In [26]:

infoscan.user_funtion


AttributeError: 'Scan2022' object has no attribute 'user_funtion'

In [ ]:
scan.ScanInfo()

In [ ]:
session.Session()

In [ ]:
Sess